In [1]:
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras import layers
from keras import models
from keras.applications.resnet50 import ResNet50
from keras.layers import Input, Conv2D, MaxPooling2D, Input, GlobalAveragePooling2D
from keras.layers import Add, Flatten, AveragePooling2D, Dense
from keras.models import Model

Using TensorFlow backend.


In [2]:
# This is one res_layer which we can use to create a deep neural network
def res_layer(input_layer, n=64):
    L1 = Conv2D(n, (3, 3), padding='same', activation='relu')(input_layer)
    L2 = Conv2D(n, (3, 3), padding='same', activation='relu')(L1)
    L2 = Conv2D(n, (3, 3), padding='same', activation='relu')(L2)
    L3 = Add()([L2, input_layer])
    return L3

In [3]:
# size of cifar10 dataset images
main_input = Input(shape=(32,32,3))
#all the layers of the res-net
L1 = Conv2D(64, (7, 7), strides=(2,2), padding='same', activation='relu')(main_input)
L2 = MaxPooling2D(pool_size=(3,3), strides=(2,2))(L1)

L2 = res_layer(L2)
L2 = res_layer(L2)
L2 = res_layer(L2)

L3 = AveragePooling2D()(L2)

L3 = Flatten()(L3)
L4 = Dense(256)(L3)
L5 = Dense(10,activation='softmax')(L4)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [4]:
#Model function from Keras shows map neural network including output shape of each layer and the number of parameters
model = Model(main_input, L5)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        9472      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
add_1 (Add)                  (None, 7, 7, 64)          0         
__________

In [5]:
#load the dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data() 
#normalize the input data
x_train = x_train.astype('float32')/255 
x_test = x_test.astype('float32')/255 
y_train = to_categorical(y_train) 
y_test = to_categorical(y_test) 
#Optimizer functino used with hyperparameters such as learning rate, decay, and momentum
sgd = optimizers.SGD(lr=0.01, decay=5e-4, momentum=0.9, nesterov=True) 
model.compile(optimizer=sgd, 
              loss='categorical_crossentropy', 
              metrics=['accuracy']) 
history = model.fit(x_train, y_train, epochs=15, shuffle='batch', batch_size=64) 
f = open('log.txt','w')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.
Epoch 1/15
50000/50000 [==============================] - 58s - loss: 1.5759 - acc: 0.4306    
Epoch 2/15
50000/50000 [==============================] - 61s - loss: 1.2113 - acc: 0.5700    
Epoch 3/15
50000/50000 [==============================] - 59s - loss: 1.0544 - acc: 0.6274    
Epoch 4/15
50000/50000 [==============================] - 63s - loss: 0.9415 - acc: 0.6678    
Epoch 5/15
50000/50000 [==============================] - 58s - loss: 0.8604 - acc: 0.6974    
Epoch 6/15
50000/50000 [==============================] - 61s - loss: 0.7896 - acc: 0.7209    
Epoch 7/15
50000/50000 [==============================] - 59s - loss: 0.7282 - acc: 0.7441    
Epoch 8/15
50000/50000 [==============================] - 62s - loss: 0.6787 - acc: 0.7604    
Epoch 9/15
50000/50000 [==============================] - 61s - loss: 0.6314 - acc: 0.7782    
Epoch 10/15
50000/50000 

In [6]:
test_loss, test_acc = model.evaluate(x_test, y_test) 
print(test_loss, test_acc)

 9856/10000 [============================>.] - ETA: 0s0.8599996905326843 0.7354


In [ ]:
# There is clearly both a high bias and high variance here -- will work to change this in the future